# Bake-off: Learning an alien language

In [1]:
__author__ = "Chris Potts"
__version__ = "CS224u, Stanford, Spring 2018 term"

## Contents

0. [Set-up](#Set-up)
0. [The doomsday scenario](#The-doomsday-scenario)
0. [The data](#The-data)
0. [Objective 1: Oracle accuracy](#Objective-1:-Oracle-accuracy)
0. [Objective 2: Predictive accuracy](#Objective-2:-Predictive-accuracy)
0. [Bake-off submission](#Bake-off-submission)
0. [Objective 3: The translation function](#Objective-3:-The-translation-function)

## Set-up

0. Make sure the `sys.path.append` value is the path to your local [SippyCup repository](https://github.com/wcmac/sippycup). (Alternatively, you can add SippyCup to your Python path; see one of the teaching team if you'd like to do that but aren't sure how.)

0. Make sure that [semparse_math_bakeoff_data.py](semparse_math_bakeoff_data.py) is in the current directory (or available via your Python path).

In [2]:
from copy import copy
import random
import sys
sys.path.append('../fork/sippycup')

## The doomsday scenario

It's an indeterminate time in the future. An alien invasion is imminent.  We have intercepted many of the aliens'  transmissions and begun the process of decoding their language. Luckily, we have found a small database of alien  language statements paired with numbers that seem to be the denotations of  those statements. 

Linguists, working tirelessly, have translated the numbers into standard arabic notation, but they have made little headway in understanding the meanings of the words and phrases in the statements. Standard bag-of-words classifiers were little help with the high-dimensional output space.

You've been called in personally by World President Zahara Jolie-Pitt-Kardashian to complete the translation task. Your goal is to use the available data to induce a lexicon mapping alien words to their associated mathematical concepts. Time is of the essence.

## The data

The data are available in `semparse_math_bakeoff_data.py`, which contains two lists of SippyCup `Example` instances:

In [3]:
from semparse_math_bakeoff_data import mathbake_train, mathbake_dev

Let's check out some examples:

In [6]:
for i in range(5):
    ex = mathbake_train[random.randint(0, len(mathbake_train))]
    print(ex.input, ex.denotation)

thouch scincs sklofg thouch sherle 1
thouch scincs sniese scwokt thouch sherle sklofg fribbs -1
scwokt volms sniese thouch kugns 2
thouch kugns sniese sherle 4
thouch kugns sklofg scwokt scincs -5


## Objective 1: Oracle accuracy

To start, the goal should be to create a grammar that can find at least one parse with the correct denotation. With that done, we can rely on features and our training data to find weights that favor the correct parses.

The other linguists on the team have extracted the vocabulary, and they can say with confidence that the words in the grammar can be classified as follows:

In [26]:
integers = ['fribbs', 'volms', 'scincs', 'kugns', 'glarc', 'sherle']
predicates = ['sniese', 'thouch', 'sklofg', 'scwokt']

We can begin building a crude grammar on this basis. We'll start with an empty one:

In [27]:
import itertools
from parsing import Grammar

# Increasing this value will increase your chances of finding 
# correct parses, but it will slow everything down.
import parsing
parsing.MAX_CELL_CAPACITY = 1000

gram = Grammar(start_symbol='$E') 

Created grammar with 0 rules


To start, we assume that the integers all have their denotations somewhere in the interval [0,5], and we consider every hypothesis of that form:

In [28]:
from parsing import Rule, add_rule

for w, i in itertools.product(integers, range(len(integers))):
    add_rule(gram, Rule('$E', w, i))

We assume also that there are unary and binary operators, so we add those combination rules:

In [29]:
# Unary connective, as in English "minus one":
add_rule(gram, Rule('$E', '$UnOp $E', lambda sems: (sems[0], sems[1])))

# First stage of binary connective, as in English "two plus":
add_rule(gram, Rule('$EBO', '$E $BinOp', lambda sems: (sems[1], sems[0])))

# Second stage of binary connective, as in English "(two plus) seven":
add_rule(gram, Rule('$E', '$EBO $E', lambda sems: (sems[0][0], sems[0][1], sems[1])))

Determining the semantic space of the operators is harder. The executor from SippyCup's `arithmetic.py` seems like a reasonable place to start:

In [30]:
unary_ops = {
    '~': lambda x: -x
}

binary_ops = {
    '+': lambda x, y: x + y,
    '-': lambda x, y: x - y,
    '*': lambda x, y: x * y
}

##################################################
#### Consider extending one or both ops dicts ####


ops = {key: val for key, val in itertools.chain(unary_ops.items(), binary_ops.items())}

def execute(semantics):
    if isinstance(semantics, tuple):
        op = ops[semantics[0]]
        args = [execute(arg) for arg in semantics[1:]]
        return op(*args)
    else:
        return semantics

__TO DO__: Bring in the words in `predicates`, in the form of a set of grammar rules like those we added for the integers. Since you don't yet know whether the predicates are unary or binary, you'll have to add rules that allow for all possible meanings.

In [34]:
##################################################
########## Add your operators rules here ######### 
for pred in predicates:
    add_rule(gram, Rule('$UnOp', pred, '~'))
    for binary_op in binary_ops.keys():
        add_rule(gram, Rule('$BinOp', pred, binary_op))

If all is going well, the vast majority of sentences of the alien language should now have a parse with a correct denotation. That is, our oracle accuracy should be at least 80%. (In fact, if it is this high, it is probably 100% but the target sometimes wasn't included in the sample of parses found during search.)

In [35]:
from parsing import parse_input

def check_oracle_accuracy(grammar=None, examples=mathbake_train, verbose=True):
    oracle = 0
    for ex in examples:
        # All the denotations for all the parses:
        dens = [execute(parse.semantics) for parse in gram.parse_input(ex.input)]
        if ex.denotation in dens:
            oracle += 1
        elif verbose:
            print("=" * 70)
            print(ex.input)
            print(set(dens))
            print(ex.denotation)
    percent_correct = int(round((oracle/float(len(examples)))*100, 0))
    print("Oracle accuracy: %s / %s (%s%%)" % (oracle, len(examples), percent_correct))

In [36]:
check_oracle_accuracy(grammar=gram, examples=mathbake_train, verbose=False)

Max cell capacity 1000 has been hit 1 times
Max cell capacity 1000 has been hit 2 times
Max cell capacity 1000 has been hit 4 times
Max cell capacity 1000 has been hit 8 times
Max cell capacity 1000 has been hit 16 times
Max cell capacity 1000 has been hit 32 times
Max cell capacity 1000 has been hit 64 times
Max cell capacity 1000 has been hit 128 times
Max cell capacity 1000 has been hit 256 times
Oracle accuracy: 196 / 200 (98%)


If your oracle accuracy isn't above 80%, then consider expanding the space of operators defined by `ops` and expanding the space of rules accordingly. __There's no guarantee that the alien language uses precisely the operators given by `ops`!__ (Hint: it would be a lot of trouble to deal with operators that could return non-`int` values.)

## Objective 2: Predictive accuracy

Your grammar is now successful in that it finds correct parses and associated denotations for the alien language. However, World President Zahara Jolie-Pitt-Kardashian is unlikely to be impressed, because you can't tell her _which_ denotation is correct, and so you can't induce a translation lexicon either. To address this, we need to find feature weights that are effective at using the training data to identify the best hypotheses allowed by the grammar.

We can start with the core features given by `scoring.rule_features`:

In [41]:
from scoring import Model, rule_features
from arithmetic import ArithmeticDomain # A source of more feature functions!

def arithmetic_features(parse):
    features = rule_features(parse)
    
    # Consider adding to the features dict based on properties of
    # parse and/or parse.semantics. SippyCup's `ArithmeticDomain`
    # has a method `operator_precedence_features` that might be
    # helpful here, for example.
    def collect_features(semantics, features):
        if isinstance(semantics, tuple):
            for child in semantics[1:]:
                collect_features(child, features)
                if isinstance(child, tuple) and child[0] != semantics[0]:
                    features[(child[0], semantics[0])] += 1.0
    collect_features(parse.semantics, features)
    
    return features

__TO DO__: Improve on the features returned by `arithmetic_features`.

Now can build and train the model.

In [42]:
model = Model(grammar=gram, feature_fn=arithmetic_features, executor=execute)

__TO DO__: Improve on the optimizer settings!

In [53]:
from learning import latent_sgd
from metrics import DenotationAccuracyMetric

##################################################
#### Consider improving the optimizer settings ###

trained_model = latent_sgd(
    model, 
    mathbake_train,
    training_metric=DenotationAccuracyMetric(), 
    T=5, 
    loss='hinge',
    l2_penalty=0.3,
    eta=0.1)

Running SGD learning on 200 examples with training metric: denotation accuracy

iter. 1; err. 208.2511380; AdaGrad mag. 1.5036566; train acc. 0.1100
iter. 2; err. 202.3809684; AdaGrad mag. 0.3814672; train acc. 0.1500
iter. 3; err. 201.2492504; AdaGrad mag. 0.2047421; train acc. 0.1750
iter. 4; err. 200.5210087; AdaGrad mag. 0.1526896; train acc. 0.1300
iter. 5; err. 199.8433145; AdaGrad mag. 0.1170238; train acc. 0.1350
iter. 6; err. 199.4795954; AdaGrad mag. 0.0937204; train acc. 0.1600
iter. 7; err. 199.2091374; AdaGrad mag. 0.0775641; train acc. 0.1550
iter. 8; err. 199.6262320; AdaGrad mag. 0.0973685; train acc. 0.0950
iter. 9; err. 198.8929657; AdaGrad mag. 0.0596885; train acc. 0.1600
iter. 10; err. 198.8621853; AdaGrad mag. 0.0581261; train acc. 0.1600
iter. 11; err. 198.7260040; AdaGrad mag. 0.0536265; train acc. 0.1600
iter. 12; err. 198.5420950; AdaGrad mag. 0.0495199; train acc. 0.1200
Max cell capacity 1000 has been hit 16384 times
iter. 13; err. 198.3315194; AdaGrad mag. 

Now that the model is trained, we can evaluate it on the held-out data:

In [54]:
from experiment import evaluate_model
from metrics import denotation_match_metrics

evaluate_model(
    model=trained_model, 
    examples=mathbake_dev, 
    examples_label="Dev",
    metrics=denotation_match_metrics(),
    print_examples=False)

Evaluating on 50 Dev examples

--------------------------------------------------------------------------------
Over 50 examples:

denotation accuracy                0.060
denotation oracle accuracy         1.000
number of parses                   814.320
spurious ambiguity                 0.678



Results:

l2 penalty = 0.1, eta = 0.5, T = 5: 0.060
l2 penalty = 0.1, eta = 0.1, T = 5: 0.120
l2 penalty = 0.5, eta = 0.1, T = 5: 0.080
l2 penalty = 0.3, eta = 0.1, T = 5: 0.160
l2 penalty = 0.3, eta = 0.1, T = 20: 0.060

## Bake-off submission

1. Enter your "denotation accuracy" score (non-oracle) from the above into the bake-off.
1. Enter a description of the feature functions and optimization settings you used.

Submission URL: https://goo.gl/forms/DCVvih3prRRa06ov2

## Objective 3: The translation function

Our primary objective was to learn how to translate the alien language into our own language for math (basic arithmetic). To see how well we did, we can look at the weights the classifier learned for the core rule-based features.

In [ ]:
from operator import itemgetter
from collections import defaultdict

def view_lexical_features(weights):
    # Get the lexical features:        
    feats = [(featname, val) for featname, val in weights.items() 
             if val > 0.0 and isinstance(featname, str) and featname.startswith('Rule')]
    # Get the core parts:
    lex = defaultdict(list)
    for featname, val in feats:
        r = eval(featname)
        lex[r.rhs[0]].append((r.sem, val))    
    # Restrict to the highest weights for each feature:
    for w, vals in lex.items():
        maxval = max([x[1] for x in vals])
        vals = [x for x in vals if x[1]==maxval]
        lex[w] = vals  
    # Printout sorted by our own semantic operators:
    for featname, vals in sorted(lex.items(), key=(lambda item: str(item[1]))):
        for val in vals:
            print("'%s' means %s (weight %0.02f)" % (featname, val[0], val[1]))

In [ ]:
view_lexical_features(trained_model.weights)

__Are you correct?__ 

[Paste in your output from view_lexical_features here to find out!](https://web.stanford.edu/class/cs224u/cgi-bin/mathbake/)

(Paste in the entire output as printed in the cell above; the script that checks the input is pretty strict about the formatting.)

This isn't part of the bake-off submission. The stakes are higher here!